# Parallelization Testing

In this notebook, I will learn how to use dask within xarray to parallelize running code and speed up parts of the Argo analysis. I'll start by running a simple test case (I hope to find) in xarray's documentation. If this work successfully, I will then move on to running the depth-->density interpolation function to see if that comes with speed improvements too.

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.path import Path
import seaborn as sns
import seaborn
import pandas as pd
import numpy as np
from importlib import reload
import cartopy.crs as ccrs
import cmocean.cm as cmo
import gsw
import dask.array as da
import dask

In [2]:
import os
os.chdir('/home.ufs/amf2288/argo-intern/funcs')
import density_funcs as df
import EV_funcs as ef
import filt_funcs as ff
import plot_funcs as pf
import processing_funcs as prf

In [3]:
reload(df)
reload(ef)
reload(ff)
reload(prf)

<module 'processing_funcs' from '/home/amf2288/argo-intern/funcs/processing_funcs.py'>

# Reproducable Test

Goal here is to make a really big array and then test loading with dask vs loading without dask. I'm following the rough steps Stephan Hoyer outlines in this blogpost (https://stephanhoyer.com/2015/06/11/xray-dask-out-of-core-labeled-arrays/), including creating a dataset with the same dimensions of

Dimensions:(latitude: 256, longitude: 512, time: 52596)

In [24]:
#from dask.distributed import Client

In [25]:
#client = Client()
#client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 9
Total threads: 72,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42894,Workers: 9
Dashboard: http://127.0.0.1:8787/status,Total threads: 72
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:37910,Total threads: 8
Dashboard: http://127.0.0.1:33734/status,Memory: 111.95 GiB
Nanny: tcp://127.0.0.1:33144,


Can also use dask gateway, then adapt(min,max) number of cores to use in the calculation

In [77]:
factor = 10

lat, lon, time = 256, 512, 52596*factor

In [78]:
#data = np.random.rand(lat,lon,time)
#data

In [79]:
data = da.random.random((time,lat,lon),chunks=(100,256,512))

In [80]:
ds = xr.Dataset(
    {
        "data": (["time", "latitude", "longitude"], data)
    },
    coords={
        "time": np.arange(time),
        "latitude": np.linspace(-90, 90, lat),
        "longitude": np.linspace(-180, 180, lon)
    }
)

In [81]:
ds

<xarray.Dataset>
Dimensions:    (time: 525960, latitude: 256, longitude: 512)
Coordinates:
  * time       (time) int64 0 1 2 3 4 5 ... 525955 525956 525957 525958 525959
  * latitude   (latitude) float64 -90.0 -89.29 -88.59 ... 88.59 89.29 90.0
  * longitude  (longitude) float64 -180.0 -179.3 -178.6 ... 178.6 179.3 180.0
Data variables:
    data       (time, latitude, longitude) float64 dask.array<chunksize=(100, 256, 512), meta=np.ndarray>

In [82]:
%time result = ds.mean('time').compute()

CPU times: user 44min 53s, sys: 10min 21s, total: 55min 14s
Wall time: 1min 4s


Okay this is a sizable dataset, with 525960*256*512 data points. It was parallelized, running on ~60 cores for the (~1min) duration of the calculation. Great! So we know dask + xarray is working here.

## Shape of Argo Data

In [4]:
#(depth*prof)/(lat*lon*time)

NameError: name 'depth' is not defined

In [5]:
depth, prof = 1000, 1000000000

In [6]:
data = da.random.random((depth,prof), chunks=(1000,10000))

In [7]:
ds = xr.Dataset({'data':(['depth','prof'],data)},
               coords={'depth':np.arange(depth),
                      'prof':np.arange(prof)})
ds

<xarray.Dataset>
Dimensions:  (depth: 1000, prof: 1000000000)
Coordinates:
  * depth    (depth) int64 0 1 2 3 4 5 6 7 8 ... 992 993 994 995 996 997 998 999
  * prof     (prof) int64 0 1 2 3 4 ... 999999996 999999997 999999998 999999999
Data variables:
    data     (depth, prof) float64 dask.array<chunksize=(1000, 10000), meta=np.ndarray>

In [25]:
mean_prof = ds.data.mean('prof')
ds_anom = ds.data - mean_prof

In [26]:
ds_anom.compute()


KeyboardInterrupt



In [29]:
def get_anom(ds, dim1='prof', variable='data'):
    mean_prof = ds[variable].mean(dim1)
    return ds[variable] - mean_prof

In [31]:
ds_mean = get_anom(ds)
ds_mean.compute()


KeyboardInterrupt



In [41]:
ds.dtypes

Frozen({'data': dtype('float64')})

In [42]:
ds_anom = xr.apply_ufunc(get_anom, ds, dask='parallelized', output_dtypes='float64')

In [44]:
ds_anom.compute()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

I wanted to see if I put only the .mean() method in a function, would it run in parallel? Yes it did. It ran on pretty much the same number of cores (50) as calling the method directly, without using a function.

Additionally, I added the anomaly portion, to have multiple operations inside the same function. This still ran in parallel.

HOWEVER, when I tried running the groupby_bins() method, this only ran on one core. My new theory: __it seems like there are some functions that automatically run in parallel and some that don't.__

So the next steps should be:
- determine which functions automatically run in parallel
- see if there are ways to parallelize functions that default to running on one core|

Notes on `apply_ufunc`:

It worked (very briefly)!!!! This is exciting, it was just about ready and running on 16 CPUs, but then an IndexError was raised: `IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices`

So I think once I can resolve that error, I'll be able to run my functions in parallel, using dask, on functions I've written :)

In [12]:
def get_MLD(
    ds, threshold=0.03, variable="SIG0", dim1="N_PROF", dim2="PRES_INTERPOLATED"
):
    """Takes an xarray and returns a new coordinate "MLD" or mixed layer depth for each profile, defined using the density threshold from the surface.
    ds: xarray with profile and pressure dimensions
    threshold: density value that defines the boundary of the mixed layer, default=0.03
    variable: density coordinate, default='SIG0'
    dim1: profile dimension, default='N_PROF'
    dim2: pressure dimension, default='PRES_INTERPOLATED'
    """

    MLD_li = []

    for n in range(0, len(ds[dim1])):
        SIG0_surface = ds.isel({dim1: n})[variable].isel({dim2: 0})
        SIG0_diff = SIG0_surface + threshold
        MLD_ds = SIG0_surface.where(ds.isel({dim1: n})[variable] < SIG0_diff)
        MLD = MLD_ds.dropna(dim2).isel({dim2: -1})[dim2]
        MLD_li.append(MLD)

    return ds.assign_coords(MLD=(dim1, MLD_li))

In [13]:
#get_MLD = dask.delayed(get_MLD)

In [ ]:
%time ds_mld = get_MLD(ds, variable='data',dim1='prof',dim2='depth').compute()

The cpu graph was all over the place for this one. It had very large dips and changes, from 65 cpus down to almost none, and then back again. Normally the cpu load stays relatively constant (within a few machines more or less) the entire time. Additionally, the usage dropped very very low about halfway through running, barely even taking up one core.

I ran this agai and it ran on 65 cpus for about a minute and then dropped down to nothing. I think the kernal disconnected?? Wondering if this is connected to the .values call?? (Trying to load something in memory but it's too large? I don't know)

Next step is to try the apply_ufunc() function to see if this can be used to parallelize an existing function I've written, such as the get_MLD function

#### For prof = 1,000,000:

- .mean(): Interestingly, there was a short spike to 20 cores for inital set-up with this ~Argo-sized dataset, however the rest of the calculation was seemingly done on 1-2. I wonder if this dataset is small enough that dask determines parallelization isn't necessary. I will now try a calculation that takes more time to see if that triggers using more cores. 

- get_MLD(): I ran this size dataset through the function to add MLD and this also only ran on 1-2 cores. Which is strange because it took a long time to run (TIME HERE). I'm wondering if this is related to how I've writen the function. For example, when I use .values within the function, and I wonder if that's going to be very slow because it loads things into memory? Does this play into anything?

- MAJOR UPDATE: it appears dask won't parallelize anything within a function automatically. So in order for me to run a function in parallel, I need to explicitly tell dask to do so using dask.delayed. Here's the documentation on this: https://examples.dask.org/delayed.html

#### For prof = 1,000,000,000

- .mean(): Okay, increasing prof to 1,000,000,000 triggers multiple cores to be used, in this case ~45. It took about 


# Argo Interpolation Test

In [10]:
atl = xr.open_dataset('/swot/SUM05/amf2288/sync-boxes/lon:(-25,-23)_lat:(-70,70)_ds_z.nc',chunks={'N_PROF':2000})

In [8]:
atl = prf.get_MLD(atl)

Calling this function only used one core. But at this point, it's hard to tell if that's because the dataset isn't big enough for dask to deem multiple cores to be necessary, or because there's something deeper going on. I'm almost wondering if I should define an argo-style dataset that's big enough to trigger multiple cores, then pass this to a function to see if this uses multiple cores

In [ ]:
print('max: {}, min: {}'.format(atl.SIG0.max().values, atl.SIG0.min().values))

In [ ]:
atl_grid = np.linspace(21,28,1000)

In [ ]:
number=np.arange(0,len(atl.N_PROF))
atl.sortby('LATITUDE')
atl.coords['N_PROF_NEW']=xr.DataArray(number,dims=atl.N_PROF.dims)

In [ ]:
%time rho_atl= df.interpolate2density_prof(atl, atl_grid)